In [1]:
from faker import Faker
from faker.providers import BaseProvider

In [2]:
import pandas as pd
import numpy as np

In [3]:
from os import path
from random import randint, randrange, choice

In [35]:
class Phone(BaseProvider):
    def phone(self):
        fmt = '1(code)number'
        code_len = randint(2, 4)
        code = [str(randint(0, 9)) for _ in range(code_len)]
        code = ''.join(code)
        number = [str(randint(0, 9)) for _ in range(7)]
        number = ''.join(number)
        return fmt.replace('code', code).replace('number', number)

In [36]:
class PassportNumber(BaseProvider):
    def passport(self):
        fmt = 'series number'
        series = [str(randint(0, 9)) for _ in range(4)]
        series = ''.join(series)
        number = [str(randint(0, 9)) for _ in range(6)]
        number = ''.join(number)
        return fmt.replace('series', series).replace('number', number)

In [49]:
class Date(BaseProvider):
    def date_(self, sy=2012):
        fmt = '{}-{:0>2}-{:0>2}'
        year = randint(sy, 2019)
        month = randint(1, 12)
        day = randint(1, 28 if month == 2 else 30)
        return fmt.format(year, month, day)

In [50]:
fake = Faker(['en_US'])
fake.add_provider(Phone)
fake.add_provider(PassportNumber)
fake.add_provider(Date)

In [51]:
def create_clients_table(count_of_entries: int) -> pd.DataFrame:
    clients = pd.DataFrame(columns=['Name', 'PhoneNumber', 'PassportId', 'Address', 'NumberOfContracts', 'TaxNumber'])
    clients['Name'] = np.array([fake.name() for _ in range(count_of_entries)])
    clients['PhoneNumber'] = np.array([fake.phone() for _ in range(count_of_entries)])
    clients['PassportId'] = np.array([fake.passport() for _ in range(count_of_entries)])
    clients['Address'] = np.array([fake.address().replace('\n', ' ').split(',')[0][:50] for _ in range(count_of_entries)])
    clients['NumberOfContracts'] = np.zeros(count_of_entries, dtype=int)
    clients['TaxNumber'] = np.array([12 + i for i in range(count_of_entries)])
    return clients

def create_agents_table(count_of_entries: int) -> pd.DataFrame:
    agents = pd.DataFrame(columns=['Name', 'PhoneNumber', 'PassportId', 'Rate', 'NumberOfContracts'])
    agents['Name'] = np.array([fake.name() for _ in range(count_of_entries)])
    agents['PhoneNumber'] = np.array([fake.phone() for _ in range(count_of_entries)])
    agents['PassportId'] = np.array([fake.passport() for _ in range(count_of_entries)])
    agents['Rate'] = np.array([randrange(0, 300, 50) for _ in range(count_of_entries)])
    agents['NumberOfContracts'] = np.zeros(count_of_entries, dtype=int)
    return agents

def create_departments_table(count_of_entries: int) -> pd.DataFrame:
    deps = pd.DataFrame(columns=['Address', 'PhoneNumber', 'DepartmentTypeID', 'NumberOfEmploees'])
    deps['Address'] = np.array([fake.address().replace('\n', ' ').split(',')[0][:50] for _ in range(count_of_entries)])
    deps['PhoneNumber'] = np.array([fake.phone() for _ in range(count_of_entries)])
    deps['DepartmentTypeID'] = np.array([randint(2, 3) for _ in range(count_of_entries)])
    deps['NumberOfEmploees'] = np.zeros(count_of_entries, dtype=int)
    return deps

def create_empcont_table(count_of_entries: int) -> pd.DataFrame:
    def position(salary: int) -> int:
        if salary >= 5000:
            return 3
        elif salary >= 4000:
            return 2
        elif salary >= 3000:
            return 1
        else:
            return 4
    
    empconts = pd.DataFrame(
        columns=['Number', 'AgentID', 'Salary', 'AgentPositionID', 'DepartmentID', 'StartDate', 'Status'])
    empconts['Number'] = np.arange(7, 7 + count_of_entries, 1, dtype=int)
    empconts['AgentID'] = np.arange(6, 6 + count_of_entries, 1, dtype=int)
    empconts['Salary'] = np.array([randrange(2000, 6000, 500) for _ in range(count_of_entries)])
    empconts['AgentPositionID'] = np.array([position(empconts['Salary'].values[i]) for i in range(count_of_entries)])
    empconts['DepartmentID'] = np.array([randint(1, 13) for _ in range(count_of_entries)])
    empconts['StartDate'] = np.array([fake.date_() for _ in range(count_of_entries)])
    empconts['Status'] = np.zeros(count_of_entries, dtype=int)
    return empconts

def create_contracts_table(count_of_entries: int) -> pd.DataFrame:
    def kind_by_obj(obj: int) -> int:
        mapping = {2: (1, 3, 5), 3: (1, 3, 5), 5: (1, 3, 5), 6: (1, 3, 5), 1: (2, 4, 7), 4: (1, 6, 7)}
        return mapping[obj]
    
    contracts = pd.DataFrame(columns=[
        'ContractKindID', 'ContractObjectID', 'InsuranceInterest', 'InsuranceAmount', 
        'Date', 'ValidityPeriod', 'ContractStatusID', 'ClientID', 'AgentID'
    ])
    contracts['ContractObjectID'] = np.array([randint(1, 6) for _ in range(count_of_entries)])
    contracts['ContractKindID'] = np.array(
        [choice(kind_by_obj(contracts['ContractObjectID'].values[i])) for i in range(count_of_entries)]
    )
    contracts['InsuranceInterest'] = np.array([randint(2, 15) for _ in range(count_of_entries)])
    contracts['InsuranceAmount'] = np.array([randrange(1500, 65000, 500) for _ in range(count_of_entries)])
    contracts['Date'] = np.array([fake.date_(2018) for _ in range(count_of_entries)])
    contracts['ValidityPeriod'] = np.array([randrange(24, 48, 12) for _ in range(count_of_entries)])
    contracts['ContractStatusID'] = np.array([1 + int(randint(0, 1000) < 150) for _ in range(count_of_entries)])
    contracts['ClientID'] = np.arange(11, 11 + count_of_entries, 1, dtype=int)
    contracts['AgentID'] = np.array([randint(1, 55) for _ in range(count_of_entries)])
    return contracts

In [52]:
test_count = 1

In [53]:
test_path_fmt = '.\\TablesExcel\\Test{}.csv'
release_path_fmt = '.\\TablesExcel\\{}.csv'

In [54]:
tables = {
    'Client': (create_clients_table, 200),
    'Agent': (create_agents_table, 50),
    'Department': (create_departments_table, 10),
    'EmploymentContract': (create_empcont_table, 50),
    'Contract': (create_contracts_table, 201)
}

In [55]:
for name, (create, cnt) in tables.items():
    test = create(test_count)
    release = create(cnt)
    
    test.to_csv(path.expandvars(test_path_fmt.format(name)), index=False)
    release.to_csv(path.expandvars(release_path_fmt.format(name)), index=False)
    
    print(f'Table {name} written')

Table Client written
Table Agent written
Table Department written
Table EmploymentContract written
Table Contract written
